# Pre-Processing

In [15]:
from data import test_data, train_data

vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print(f'{vocab_size} unique words found')

word_to_idx = { w: i for i, w in enumerate(vocab)}
idx_to_word = { i: w for i, w in enumerate(vocab)}
print(word_to_idx['sad'])
print(idx_to_word[0])

18 unique words found
3
and


In [16]:
import numpy as np

def createInputs(text:str):
    inputs = []
    for w in text.split(' '):
        v = np.zeros((vocab_size, 1))
        v[word_to_idx[w]] = 1
        inputs.append(v)
    return inputs


# RNN Setup class

In [17]:
import numpy as np
from numpy.random import randn

class RNN:
    def __init__(self, input_size, output_size, hidden_size=64):
        # Weights
        
        self.Whh = randn(hidden_size, hidden_size) / 1000
        self.Wxh = randn(hidden_size, input_size) / 1000
        self.Why = randn(output_size, hidden_size) / 1000
        
        # Biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs):
        h = np.zeros((self.Whh.shape[0], 1))
        
        self.last_inputs = inputs
        self.last_hs = { 0: h }
        for i, x in enumerate(inputs):
           h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
           self.last_hs[i + 1] = h
        y = self.Why @ h + self.by
        
        return y, h
    
    def backprop(self, d_y, learn_rate=2e-2):
        n = len(self.last_inputs)
        
        d_Why = d_y @ self.last_hs[n].T
        d_by = d_y
        
        d_Whh = np.zeros(self.Whh.shape)
        d_Wxh = np.zeros(self.Wxh.shape)
        d_bh = np.zeros(self.bh.shape)
        
        d_h = self.Why.T @ d_y
        
        for t in reversed(range(n)):
            temp = ((1 - self.last_hs[t + 1] ** 2)* d_h)
            d_bh += temp
            d_Whh += temp @ self.last_hs[t].T
            
            d_Wxh += temp @ self.last_inputs[t].T
            d_h = self.Whh @ temp
            
            for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
                np.clip(d, -1, 1, out=d)
                
            self.Whh -= learn_rate * d_Whh
            self.Wxh -= learn_rate * d_Wxh
            self.Why -= learn_rate * d_Why
            self.bh -= learn_rate * d_bh
            self.by -= learn_rate * d_by
            
def softmax(xs):
    return np.exp(xs) / sum(np.exp(xs))
    
rnn = RNN(vocab_size, 2)
inputs = createInputs('i am very good')
out, h = rnn.forward(inputs)
probs = softmax(out)
print(probs)

[[0.49999773]
 [0.50000227]]


# Setup

In [18]:
for x, y in train_data.items():
    inputs = createInputs(x)
    target = int(y)
    
    # forward 
    out, _ = rnn.forward(inputs)
    prob = softmax(out)
    
    d_L_d_y = probs
    d_L_d_y[target] -= 1
    
    # backward
    rnn.backprop(d_L_d_y)

# Testing

In [19]:
import random

def processData(data, backprop=True):
    items = list(data.items())
    random.shuffle(items)
    
    loss = 0
    num_correct = 0
    
    for x,y in items:
        inputs = createInputs(x)
        target = int(y)
        
        #Forward
        out, _ = rnn.forward(inputs)
        probs = softmax(out)
        
        # Loss
        loss -= np.log(prob[target])
        num_correct += int(np.argmax(probs) == target)
        
        if backprop:
            d_L_d_y = probs
            d_L_d_y[target] -= 1
            
            rnn.backprop(d_L_d_y)
    
    return loss/len(data), num_correct/len(data)

# Training loop

In [20]:
for epoch in range(1000):
    train_loss , train_acc = processData(train_data)
    
    if epoch % 100 == 99:
        print("-- Epoch %d" % (epoch + 1))
        print("Train:\nLoss %.3f | Accuracy: %.3f" % (train_loss, train_acc))
        
        test_loss, test_acc = processData(test_data, backprop=False)
        print("Test:\nLoss: %.3f | Accuracy: %.3f" % (test_loss, test_acc))

-- Epoch 100
Train:
Loss 1.117 | Accuracy: 0.569
Test:
Loss: 1.235 | Accuracy: 0.650


-- Epoch 200
Train:
Loss 1.117 | Accuracy: 0.828
Test:
Loss: 1.235 | Accuracy: 0.900
-- Epoch 300
Train:
Loss 1.117 | Accuracy: 1.000
Test:
Loss: 1.235 | Accuracy: 1.000
-- Epoch 400
Train:
Loss 1.117 | Accuracy: 1.000
Test:
Loss: 1.235 | Accuracy: 1.000
-- Epoch 500
Train:
Loss 1.117 | Accuracy: 1.000
Test:
Loss: 1.235 | Accuracy: 1.000
-- Epoch 600
Train:
Loss 1.117 | Accuracy: 1.000
Test:
Loss: 1.235 | Accuracy: 1.000
-- Epoch 700
Train:
Loss 1.117 | Accuracy: 1.000
Test:
Loss: 1.235 | Accuracy: 1.000
-- Epoch 800
Train:
Loss 1.117 | Accuracy: 1.000
Test:
Loss: 1.235 | Accuracy: 1.000
-- Epoch 900
Train:
Loss 1.117 | Accuracy: 1.000
Test:
Loss: 1.235 | Accuracy: 1.000
-- Epoch 1000
Train:
Loss 1.117 | Accuracy: 1.000
Test:
Loss: 1.235 | Accuracy: 1.000
